## UNIVERSIDAD POLITECNICA SALESIANA

**Integrantes:** Pedro Bermeo, María José Peláez

**Asignatura:** Inteligencia Artificial II

### Visualizador e Identificador de Digitos

A continuación se detallará el proceso que se tuvo para el desarrollo del visualizador de digitos y los resultados 
que se obtuvieron.

### Explicación Código

* El código a continuación permite leer el corpus con la información de los números que se pretende visualizar en la pantalla de la matriz.

* El método **generate_indices()** permite generar los indices al abrir el dataset.
* El método **get_digit()** permite obtener los digitos del dataset para posteriormente listarlos en el combobox que se visualiza en la pantalla.

In [ ]:
import re
import itertools

class Utilities:
    
    def __init__(self, path = './corpus/digits-database.data'):
        self.path = path
        self.regex = re.compile('(0|1){2,}')
        self.regexno = re.compile('(\s)+[0-9]{1}')
        
    
    def generate_indices(self):
        _dict = []
        with open(self.path, 'r') as _f:
            pivote = 0
            flag = False
            lineno = 0
            for line in _f:
                if self.regex.match(line)!=None and not flag:
                    pivote = lineno
                    flag = True
                if self.regexno.match(line)!=None and flag:
                    _dict.append((int(line.replace(' ','')),pivote,lineno))
                    flag = False
                lineno += 1
            _f.close()
            
        return _dict

    def get_digit(self,_slice, _end):
        data = []
        with open(self.path, 'r') as _f:
            for line in itertools.islice(_f, _slice, _end):
                data.append([int(i) for i in line.lstrip().rstrip()])
            
            _f.close()
        return data

* El método **obtenerCodificacion** permite codificar y transformar mediante OneHotEncoder la información recibida.

* El método **obtenerEntrenamiento** permite entrenar a la red neuronal en este caso se utiliza 1000 iteraciones con la activacion **logistic**

* También una vez reconocido el número dibujado en pantalla permite reemplazar los ceros por los unos dependiendo al número de celdas que se utilizaron.

* Posteriomente se procede a imprimir la matriz de confusión con el resultado de las precisiones y el promedio total.

In [ ]:
	def obtenerCodificacion(self):
		salida = self.label_encoder.fit_transform(self.delta)
		onehot_encoder = OneHotEncoder(sparse=False)
		salida = salida.reshape(len(salida), 1)
		self.onehot_encoded = onehot_encoder.fit_transform(salida)
	
	def obtenerEntrenamiento(self):
	
		x_train, x_test, d_train, d_test = train_test_split(self.datos, self.onehot_encoded, test_size=0.80, random_state=0)
		mlp = MLPClassifier(solver = 'lbfgs', activation='logistic', verbose=True, alpha=1e-4, tol=1e-15, max_iter=1000, \
		hidden_layer_sizes=(1024, 512, 256, 128, 10))
		mlp.fit(self.datos, self.onehot_encoded)
		
		for entrada in self.datos:
			res = mlp.predict(entrada.reshape(1, -1))
			num = (np.argmax(res, axis=1)+1).reshape(-1, 1)

			aux = []
			matriz = []
			for i in range(32):
				for j in range(i * 32, (i + 1) * 32):
					aux.append(entrada[j])
				matriz.append(aux)
				aux = []

			for i in range(32):
				print(str(matriz[i]).replace(', ', ''))
			print(res, '=>',int(num[0] - 1))

		prediccion = (np.argmax(mlp.predict(x_test), axis = 1) + 1).reshape(-1, 1)
		print('Matriz de Confusion\n')
		matriz = confusion_matrix((np.argmax(d_test, axis = 1) + 1).reshape(-1, 1), prediccion)
		print(matriz)
		print('\n')
		print(classification_report((np.argmax(d_test, axis = 1) + 1).reshape(-1, 1), prediccion))

* El método que se visualiza a continuación permite cargar el dataset, el cual realizamos la lectura anteriormente, obteniendo cada uno de los dígitos y visualizandolos en pantalla para poder realizar la selección correspondiente y que se pueda observar en pantalla el reconocimiento.

In [ ]:
def cargaDatos(self, evt):
		self.renovar()
		_id = [int(i) for i in (str(self.cbxData.get()).replace(')','').replace(' ','').replace('(','')).split(',')]
		
		data = self.utilities.get_digit(_id[1],_id[2])
		for i in range(len(data)):
			for j in range(len(data[i])):
				if(data[i][j] == 1):
					self.seleccionar(j, i)

1. La interfaz del visualizador se presenta a continuación:
    
 ![Texto Alternativo](visualizador.png) 


2. Al hacer clic en el botón denominado **Entrenamiento** la función que permite realizar es el entrenamiento de la 
red Neuronal y permite observar en consola el resultado del entrenamiento y la matriz de confusión.

**Iteraciones**

![Texto Alternativo](b.png) 

**Matriz de Confusión**

![Texto Alternativo](a.png) 

A continuación se puede observar la tabla con todas las precisiones y el promedio de estas.

![Texto Alternativo](d.png) 

3. Al hacer clic en el botón denominado **Nuevo** permite limpiar la matriz para volver a dibujar un número.

4. Al hacer clic en el botón denominado **Dibujar Numero** permite una vez que se haya dibujado un número en la pantalla obtener la predicción y me permita obtener que número es el que dibuje eso mediante el entrenamiento que 
tuvo la red neuronal para que sea mas preciso.

### Resultados:

* Vamos a dibujar en la matriz el número uno y observaremos el resultado que nos devuelve la predicción de la red neuronal.
* El resultado que se obtuvo también fue uno, entonces la predicción es correcta, como se observa en la imagen.

![Texto Alternativo](e.png) 

* Se puede observar que en la consola los unos representa lo que se dibujó en pantalla y los ceros representa las celdas que no fueron utilizadas.

![Texto Alternativo](c.png) 

* El resultado que se visualiza a continuación es de la lectura del dataset, el cual se hizo la lectura de los datos y se visualiza en pantalla el número que corresponde del dataset que hayamos seleccionado mediante el combobox.

![Texto Alternativo](f.png) 
